In [16]:
import os
import requests
from bardapi import BardCookies
from sydney import SydneyClient
from dotenv import load_dotenv
import markdown
import re
import random
import string

load_dotenv()

True

In [3]:
models = [
    ('summarize', 'facebook/bart-large-cnn'),
    ('summarize', 'google/pegasus-large'),
    ('summarize', 'marianna13/flan-t5-base-summarization'),
    ('toxic', 's-nlp/roberta_toxicity_classifier'),
    ('toxic', 'citizenlab/distilbert-base-multilingual-cased-toxicity', 'inputs'),
    ('toxic', 'martin-ha/toxic-comment-model'),
    ('spam', 'rafacost/bert_base_pt_en_cased_email_spam'),
    ('spam', 'h-e-l-l-o/email-spam-classification-merged'),
    ('spam', 'dima806/email-spam-detection-roberta'),
    ('translate', 't5-base'),
    ('translate', 'allenai/wmt16-en-de-12-1'),
    ('translate', 'facebook/wmt19-en-de'),
    ('fillmask', 'bert-base-uncased', '[MASK]'),
    ('fillmask', 'vinai/bertweet-base', '<mask>'),
    ('fillmask', 'roberta-base', '<mask>'),
]

In [4]:
def request_to_model(model, input):

    BASE_URL = "https://api-inference.huggingface.co/models/"
    headers = {"Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_KEY')}"}
    new_input = input
    parameters = {}

    if model[0] == 'fillmask':
        new_input = input.replace('<mask>', model[2])
    elif model[0] == 'translate':
        parameters = {"src_lang": "en_XX", "tgt_lang": "tgt_XX"}

    query = {"inputs": new_input, "wait_for_model": True, "parameters":parameters}

    try:
        response = requests.post(BASE_URL + model[1], json=query, headers=headers)
    except Exception as exception:
        print(exception)
    
    return response.json()

In [9]:
res = request_to_model(models[10], "The tower is 324 metres (1,063 ft) tall") # "The goal of life is <mask>."
print(res)

[{'translation_text': 'Das Handtuch ist 324 Meter groß.'}]


In [10]:
def convert_to_plain_text(input_text):
    input_text = re.sub(r'\[\^\d+\^\]', '', input_text)
    plain_text = markdown.markdown(input_text, output_format='html')
    plain_text = re.sub(r'<[^>]*>\s*', '', plain_text)
    return plain_text

In [11]:
async def request_to_bing(question, type="q&a"):

    prompt = ""

    if (type == "q&a"):
        prompt = "Answer me the following question in plain text without using quotes: "
    elif (type == "change_order"):
        prompt = "Change the order of the next sentence: "

    async with SydneyClient() as sydney:
        response = await sydney.ask(prompt + question, citations=False)
        response = convert_to_plain_text(response)
        return response

In [12]:
response = await request_to_bing("When was Bing Chat released?")
print(response)

Bing Chat was released on February 8th, 2023.


In [88]:
response = await request_to_bing("How many people live in Berlin? I live in Spain", "change_order")
print(response)

Sure, I can help you with that. Here's the sentence with the order changed: "I live in Spain. How many people live in Berlin?" I hope that helps!.


In [10]:
def request_to_bard(question):

    cookie_dict = {
        "__Secure-1PSID": os.getenv('SECURE_1PSID'),
        "__Secure-1PSIDTS": os.getenv('SECURE_1PSIDTS'),
        "__Secure-1PSIDCC": os.getenv('SECURE_1PSIDCC'),
    }

    bard = BardCookies(cookie_dict=cookie_dict)
    response = bard.get_answer(question)['content']
    return response

In [13]:
print(request_to_bard("When was Bard released?"))

Bard was initially announced on February 17, 2023, and released in a limited capacity on March 21, 2023. It was upgraded to PaLM and then to Gemini in 2023.


PERTURBACIONES

In [27]:
def sustituir_caracteres(cadena, nivel):
    if nivel < 1 or nivel > 10:
        return "El nivel debe estar entre 1 y 10."
    
    caracteres = list(cadena)
    indices_a_sustituir = [i for i in range(len(caracteres)) if caracteres[i] in string.ascii_letters]
    num_caracteres_a_sustituir = int(len(indices_a_sustituir) * nivel / 10)
    indices_a_sustituir = random.sample(indices_a_sustituir, num_caracteres_a_sustituir)
    
    for indice in indices_a_sustituir:
        caracteres[indice] = random.choice(string.ascii_letters)
    
    return ''.join(caracteres)


In [32]:
print(sustituir_caracteres("Hola, ¿cómo estás? Ha pasado bastante tiempo desde que tu y yo nos conocimos", 1))

Hola, ¿Hómo estág? Ha pasado bastante tiempe desde que tu y yo nSs conocjmos
